# Hand-made Convolutional Neural Network
In this notebook both the features extractor and the classifier are manually built.

Import the needed libraries and connect Drive:

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import os
import tensorflow as tf
import numpy as np
import shutil
 
SEED = 1518
tf.random.set_seed(SEED)  
 
# Get current working directory
cwd = os.getcwd()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Get dataset directory
dataset_dir = os.path.join(cwd, 'drive/My Drive/MaskDataset')
dataset_dir
train_dir = os.path.join(dataset_dir,'training')
os.listdir(dataset_dir)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
 
apply_data_augmentation = True
 
# Create training ImageDataGenerator object
if apply_data_augmentation:
    train_data_gen = ImageDataGenerator(rotation_range=5,
                                        width_shift_range=10,
                                        height_shift_range=10,
                                        zoom_range=0.3,
                                        horizontal_flip=True,
                                        vertical_flip=False,
                                        fill_mode='constant',
                                        cval=0,
                                        rescale=1./255,
                                        validation_split=0.15)
else:
    train_data_gen = ImageDataGenerator(rescale=1./255,
                                        validation_split=0.15)

In [ ]:
# Batch size
bs = 8
 
# Image shape (forced)
img_width = 256
img_height = 256
 
# Classes
num_classes = 3
 
# Generate training & validation
train_gen = train_data_gen.flow_from_directory(train_dir,
                                               batch_size=bs,
                                               classes=None,
                                               class_mode='categorical', 
                                               color_mode='rgb',        
                                               shuffle=True,
                                               subset='training',
                                               seed=SEED) 
valid_gen = train_data_gen.flow_from_directory(train_dir,
                                               batch_size=1,
                                               classes=None,
                                               class_mode='categorical', 
                                               color_mode='rgb',        
                                               shuffle=True,
                                               subset='validation',
                                               seed=SEED)
 
 
# Training set creation (85pc) and valid set creation(15pc)
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_height, img_width, 3], [None, num_classes]))
valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_height, img_width, 3], [None, num_classes]))
 
# Repeat()
train_dataset.repeat()
valid_dataset.repeat()

Build the model

In [ ]:
# Featuers extractor + classifier
 
start_f = 64  #initial number of filters
depth = 4    #number of convolutional layers
 
model = tf.keras.Sequential()
 
# Features extractor
for i in range(depth):
 
    if i == 0:
        input_shape = [img_height, img_width, 3]
    else:
        input_shape=[None]
 
    # Conv block: Conv2D -> Activation -> Pooling
    model.add(tf.keras.layers.Conv2D(filters=start_f, 
                                     kernel_size=(3, 3),
                                     strides=(1, 1),
                                     padding='same',
                                     input_shape=input_shape))
    model.add(tf.keras.layers.ReLU())
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))      
 
    start_f *= 2                                                
    
# Classifier

average_pool = tf.keras.Sequential()
average_pool.add(tf.keras.layers.AveragePooling2D())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))
 
# Visualize created model as a table
model.summary()


Set optimization parameters and compile the model

In [ ]:
# Loss
loss = tf.keras.losses.CategoricalCrossentropy()
 
# learning rate
lr = 1e-4
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
 
# Validation metrics
metrics = ['accuracy']
 
# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
#%reload_ext tensorboard
#%tensorboard --logdir /content/drive/My\ Drive/MaskDataset/classification_experiments --port 8008

In [ ]:
# Callbacks
import os
from datetime import datetime
 
 
cwd = os.getcwd()
 
exps_dir = os.path.join('/content/drive/My Drive/MaskDataset/', 'classification_experiments_HM')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)
 
now = datetime.now().strftime('%b%d_%H-%M-%S')
 
model_name = 'CNN_1'
 
exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
callbacks = []
 
# Visualize Learning on Tensorboard
tb_dir = os.path.join(exp_dir, 'tb_logs')
if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)
    
# Shows losses and metrics for both training and validation
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                             profile_batch=0,
                                             histogram_freq=1)  
callbacks.append(tb_callback)
 


Apply Early Stopping

In [ ]:
# Early Stopping

early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    callbacks.append(es_callback)

In [ ]:
# Model fitting
 
model.fit(x=train_dataset,
          epochs=80,  
          steps_per_epoch=len(train_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_gen), 
          callbacks=callbacks)

Epoch 1/80
597/597 [==============================] - 118s 198ms/step - loss: 1.1002 - accuracy: 0.3421 - val_loss: 1.0945 - val_accuracy: 0.3757
Epoch 2/80
597/597 [==============================] - 118s 198ms/step - loss: 1.0865 - accuracy: 0.3918 - val_loss: 1.0701 - val_accuracy: 0.3912
Epoch 3/80
597/597 [==============================] - 118s 198ms/step - loss: 1.0194 - accuracy: 0.4810 - val_loss: 0.9319 - val_accuracy: 0.5327
Epoch 4/80
597/597 [==============================] - 118s 198ms/step - loss: 0.8897 - accuracy: 0.5724 - val_loss: 0.8437 - val_accuracy: 0.6088
Epoch 5/80
597/597 [==============================] - 118s 198ms/step - loss: 0.8039 - accuracy: 0.6181 - val_loss: 0.8700 - val_accuracy: 0.5612
Epoch 6/80
597/597 [==============================] - 118s 198ms/step - loss: 0.7277 - accuracy: 0.6583 - val_loss: 0.7264 - val_accuracy: 0.6552
Epoch 7/80
597/597 [==============================] - 119s 199ms/step - loss: 0.6877 - accuracy: 0.6769 - val_loss: 0.6987 -

In [ ]:
#tensorboard --logdir /content/drive/MyDrive/MaskDataset/classification_experiments/ --port 6009 

Test the model and export results

In [ ]:
import os
from PIL import Image
cwd = os.getcwd()

dir = os.path.join('/content/drive/My Drive/Progetto/', 'test')
results = {}
for subdir, dirs, files in os.walk(dir):
    for file in files:
        filepath = subdir + os.sep + file
        img = Image.open(filepath).convert('RGB')
        img =img.resize((img_height,img_width))
        img_array = np.array(img)
        img_array = np.expand_dims(img_array, 0) 
        img_tensor=tf.convert_to_tensor(img_array)
        img_tensor = tf.cast(img_array,tf.float32)/255.
        softmax=model(img_tensor)
        prediction = np.argmax(softmax)   # predicted class
        results[file] = prediction
        
        

        

In [ ]:
results

In [ ]:
import os
from datetime import datetime

def create_csv(results, results_dir='/content/drive/My Drive/MaskDataset/Results'):

    csv_fname = 'results_inception_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')


In [ ]:
create_csv(results)